In [1]:
from opentrons import protocol_api
from opentrons import simulate

In [2]:
# input plate
# Columns 1 to 4 have the gRNA primers for assembly with 1 being for 
# the crispr expression vector directly or for subarray 1
# col 2 for subarray 2
# col 3 for sub array 3
# col 4 for subarray 4
# Rows A to C for activation vectors and D to F for repression vectors

In [3]:
from opentrons import simulate

metadata = {
    'apiLevel': '2.15',
    'author': 'Hafsa Kaja Moinudeen',
    'protocolName': 'golden gate assembly of gRNA expression cassettes for CRISPR multiplexing',
    'source': 'Automated protocol from PMC9411621',
    'category': 'Synthetic Biology',
    'createdAt': '2023-12-08',
    'lastModified': '2023-12-08'
}

protocol = simulate.get_protocol_api('2.15')

# Labware
source_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 3)
rxn_plate = protocol.load_labware('corning_96_wellplate_360ul_flat', 2)
tiprack_1 = protocol.load_labware('opentrons_96_tiprack_20ul', 5)
tiprack_2 = protocol.load_labware('opentrons_96_tiprack_20ul', 6)
thermal_cycler_module = protocol.load_module(module_name='thermocyclerModuleV2')  # Specify the correct labware and position
thermal_cycler_plate = thermal_cycler_module.load_labware(name='nest_96_wellplate_100ul_pcr_full_skirt')

temp_mod = protocol.load_module(module_name="temperature module gen2", location = 'D1')
temp_plate = temp_mod.load_labware('opentrons_24_aluminumblock_nest_1.5ml_snapcap') 
# this is where the cold stuff is going to go
# Pipettes
p20_single = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tiprack_1, tiprack_2])  # Changed to 'right'

C:\Users\hafsa\.opentrons\robot_settings.json not found. Loading defaults
Deck calibration not found.
C:\Users\hafsa\.opentrons\deck_calibration.json not found. Loading defaults


In [4]:
# master mix 
# activation contents (+ 10%)
# Q5 rxn buffer = 4ul * num of activation gRNAs
# dNTP = 0.4ul * num of activation gRNAs
# DNA polymerase = 0.2 ul * num of activation gRNAs
# H20 = 13.4 ul * num of activation gRNAs
# means each rxn will need 18 ul

# repression contents (+ 10%)
# Q5 rxn buffer = 4ul * num of repression gRNAs
# dNTP = 0.4ul * num of repression gRNAs
# DNA polymerase = 0.2 ul * num of repression gRNAs
# pWS3799 plasmid = 1ul * num of repression grnas
# H20 = 12.4 ul * num of repression gRNAs
# means each rxn will need 18 ul

In [6]:
num_of_arrays =  4 # number of subarrays you are running
# below wells can be in any order/ number, however if you are doing mutiple arrays, they should be symmetrical
activation_gRNA_wells = 'ABC'
repression_gRNA_wells = 'DEF'

In [7]:
# generating gRNA fragments using PCR - 6 only
# Set the temperature (e.g., 37 degrees Celsius)
temp_mod.set_temperature(4)

# Wait for the Temperature Module to reach the set temperature
temp_mod.await_temperature(4)

In [8]:
for j in range(1, num_of_arrays+1):
    # activation
    # add the activation master mix to the three primer-pairs containing wells (hand pippetted)
    # put it in the thermocycler for 5 cycles
    p20_single.pick_up_tip()
    for i in activation_gRNA_wells:
        p20_single.transfer(18, temp_plate['A6'], source_plate[f'{i}{j}'], new_tip='never')
    p20_single.drop_tip()

    # repression
    # add the activation master mix to the three primer/plasmid containing wells (hand pippetted)
    # put it in the thermocycler for 30 cycles
    p20_single.pick_up_tip()
    for i in repression_gRNA_wells:
        p20_single.transfer(18, temp_plate['B6'], source_plate[f'{i}{j}'], new_tip='never')
    p20_single.drop_tip()
    # after 25 cycles, open thermocycler and put the the activation gRNA in pcr plate in the thermocycler for an additional 5 cycles

In [9]:
thermal_cycler_module.open_lid()
for j in range(1, num_of_arrays+1):
    # addition of activation gRNA to the PCR plate
    for i in repression_gRNA_wells:
        p20_single.transfer(20, source_plate[f'{i}{j}'], thermal_cycler_plate[f'{i}{j}'])
thermal_cycler_module.close_lid()

'closed'

In [10]:
# Start the initial 25 cycles of the repression reaction
thermal_cycler_module.set_lid_temperature(105)  # Set the lid temperature, adjust as needed

# Initial denaturation step
thermal_cycler_module.set_block_temperature(98, hold_time_seconds=30)

# Cycling steps (25 cycles)
for _ in range(25):
    thermal_cycler_module.set_block_temperature(98, hold_time_seconds=10)
    thermal_cycler_module.set_block_temperature(57, hold_time_seconds=20)
    thermal_cycler_module.set_block_temperature(72, hold_time_seconds=30)

In [11]:
# After 25 cycles, open the lid for manual addition of activation gRNA
thermal_cycler_module.open_lid()

for j in range(1,  num_of_arrays+1):
    # addition of activation gRNA to the PCR plate
    for i in activation_gRNA_wells:
        p20_single.transfer(20, source_plate[f'{i}{j}'], thermal_cycler_plate[f'{i}{j}'])
    # Close the lid to continue with the remaining 5 cycles
thermal_cycler_module.close_lid()

'closed'

In [12]:
# Additional denaturation step before the final 5 cycles
thermal_cycler_module.set_block_temperature(98, hold_time_seconds=30)

# Final 5 cycles with different temperature settings
for _ in range(5):
    thermal_cycler_module.set_block_temperature(98, hold_time_seconds=10)
    thermal_cycler_module.set_block_temperature(61, hold_time_seconds=20)
    thermal_cycler_module.set_block_temperature(72, hold_time_seconds=30)

# Final denaturation step
thermal_cycler_module.set_block_temperature(98, hold_time_seconds=30)

# Hold at 10 °C
thermal_cycler_module.set_block_temperature(10, hold_time_minutes=10)  # 0 seconds means hold indefinitely

# Open the lid at the end
thermal_cycler_module.open_lid()


'open'

'ABCDEF'

In [13]:
# Insert a pause
protocol.pause('Protocol paused. Please resume after manual steps are completed. PCR purification using gel')

In [16]:
# golden gate assembly
# input - we have the purified fragments in col 6-9 A to F  
# CRISPR vector or subarray
thermal_cycler_module.open_lid()

for j in range(6, num_of_arrays+6):
    for i in activation_gRNA_wells + repression_gRNA_wells:
        p20_single.transfer(1, source_plate[f'{i}{j}'], thermal_cycler_plate[f'A{j}'])
        
# total volume of reaction must be twenty
volume_for_gg_1 = 20 - len(activation_gRNA_wells + repression_gRNA_wells)
    
# transfer master mix from C1, C2, C3, and C4 to the thermal cycler_plate directly 

for j in range(6, num_of_arrays+6): 
    tar = j-5
    p20_single.transfer(volume_for_gg_1, temp_plate[f'C{tar}'], thermal_cycler_plate[f'A{j}'])

thermal_cycler_module.close_lid()

'closed'

In [17]:
# Set the thermal cycling parameters
thermal_cycler_module.set_lid_temperature(105)  # Set the lid temperature, adjust as needed

# Run the thermal cycling program
# (37 °C for 5 min, 16 °C for 5 min) x 30 cycles
for _ in range(30):
    thermal_cycler_module.set_block_temperature(37, hold_time_minutes=5)
    thermal_cycler_module.set_block_temperature(16, hold_time_minutes=5)

# Final digestion step at 55 °C for 10 min
thermal_cycler_module.set_block_temperature(55, hold_time_minutes=10)

# Heat inactivation at 80 °C for 10 min
thermal_cycler_module.set_block_temperature(80, hold_time_minutes=10) 

# Hold at 10 °C
thermal_cycler_module.set_block_temperature(10, hold_time_minutes=10) 

# open the lid
thermal_cycler_module.open_lid()


'open'

In [20]:
protocol.pause('Protocol paused. Please resume after manual steps are completed. Transformation and miniprep')

In [21]:
# transformation - do manually
# select colonies that do not floresce

In [22]:
# mini prep

##  BsmBI Golden Gate assembly of subarrays
##  Master mix
0.5 μL of CRISPRai vector/sub-array plasmid (50 fmol/μL)

1 μL of each sub-array/spacer (50 fmol/μL)

1 μL of T4 DNA ligase buffer (NEB)

0.5 μL of T4 DNA ligase (NEB)

0.5 μL of BsmBI v2 (NEB)

7.5 - num_of_arrays µL of ddH2O. 

### Purified subarrays in column 11 from A to D

In [23]:
def generate_alphabet_string(n):
    # Generate a string of alphabets in uppercase depending on the input number
    # A = 1, AB = 2, ABC = 3, and so on
    alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    return alphabet[:n]

In [24]:
# wells containing the subarrays 
subarray_wells = generate_alphabet_string(num_of_arrays)

# final assembly
thermal_cycler_module.open_lid()
# transfer all subarrays to the thermal cycler
for i in subarray_wells:
    p20_single.transfer(1, source_plate[f'{i}11'], thermal_cycler_plate['A11'])

# total volume of reaction must be ten
volume_for_gg_2 = 10 - num_of_arrays
 
# transfer mastermix D6
p20_single.transfer(volume_for_gg_2, temp_plate['D6'], thermal_cycler_plate['A11'])   
thermal_cycler_module.close_lid()

'closed'

In [25]:
# Set the thermal cycling parameters
thermal_cycler_module.set_lid_temperature(105)  # Set the lid temperature, adjust as needed

# Run the thermal cycling program
# (42 °C for 2 min, 16 °C for 5 min) x 25 cycles
for _ in range(25):
    thermal_cycler_module.set_block_temperature(42, hold_time_minutes=2)
    thermal_cycler_module.set_block_temperature(16, hold_time_minutes=5)

# Final digestion step at 55 °C for 10 min
thermal_cycler_module.set_block_temperature(55, hold_time_minutes=10)

# Heat inactivation at 80 °C for 10 min
thermal_cycler_module.set_block_temperature(80, hold_time_minutes=10)

# Hold at 10 °C
thermal_cycler_module.set_block_temperature(10, hold_time_minutes=10)  

# Optionally, open the lid if your next step requires it
thermal_cycler_module.open_lid()

'open'

In [26]:
protocol.pause('Protocol complete. Team OT-5 are genius!!!')

In [27]:
#print out commands so we can check the simulation
for line in protocol.commands(): 
        print(line)

Setting Temperature Module temperature to 4.0 °C (rounded off to nearest integer)
Waiting for Temperature Module to reach temperature 4.0 °C (rounded off to nearest integer)
Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on slot 5
Transferring 18.0 from A6 of Opentrons 24 Well Aluminum Block with NEST 1.5 mL Snapcap on Temperature Module GEN2 on slot 1 to A1 of Corning 96 Well Plate 360 µL Flat on slot 3
Aspirating 18.0 uL from A6 of Opentrons 24 Well Aluminum Block with NEST 1.5 mL Snapcap on Temperature Module GEN2 on slot 1 at 7.56 uL/sec
Dispensing 18.0 uL into A1 of Corning 96 Well Plate 360 µL Flat on slot 3 at 7.56 uL/sec
Transferring 18.0 from A6 of Opentrons 24 Well Aluminum Block with NEST 1.5 mL Snapcap on Temperature Module GEN2 on slot 1 to B1 of Corning 96 Well Plate 360 µL Flat on slot 3
Aspirating 18.0 uL from A6 of Opentrons 24 Well Aluminum Block with NEST 1.5 mL Snapcap on Temperature Module GEN2 on slot 1 at 7.56 uL/sec
Dispensing 18.0 uL into B1 of Corning 9